In [ ]:
import pandas as pd
import numpy as np
import matplotlib as plt
import seaborn as sns
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

In [ ]:
df = pd.read_csv(data)
df.shape
df.head()
df.info()
df.summary()
df.isnull().sum()

### X y

In [ ]:
X = df.drop('Channel', axis = 1)
y = df['Channel']

# convert labels into binary values
y[y == 2] = 0
y[y == 1] = 1

### train

In [1]:
# convert dataset into an optimized data structure called Dmatrix that XGBoost supports
import xgboost as xgb
data_dmatrix = xgb.DMatrix(data = X, label = y)

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 0)

params = {
    'objective':'binary:logistic',  # for classification problems with probability
    'max_depth':4, 
    'alpha':10,  # l1 regularization
    'learning_rate':1.0,  # [0,1]
    'n_estimators':100
}
# subsample, colsample_bytree, gamma, lamda (l2 regularization)

xgb_clf = xgb.XGBClassifier(**params)
xgb_clf.fit(X_train, y_train)

C:\Users\lenovo\Anaconda3\lib\site-packages\dask\dataframe\utils.py:13: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


### test and evalution

In [ ]:
y_pred = xgb_clf.predict(X_test)

from sklearn.metrics import accuracy_score
print('XGBoost model accuracy score: {0:0.4f}'.format(accuracy_score(y_test, y_pred)))

### k-fold cv

In [ ]:
params = {"objective":"binary:logistic",
          'colsample_bytree': 0.3,
          'learning_rate': 0.1,
          'max_depth': 5, 
          'alpha': 10}

xgb_cv = xgb.cv(dtrain = data_dmatrix,
                params = params,
                nfold = 3,  # k-fold
                num_boost_round = 50,  # number of trees
                early_stopping_rounds = 10,
                metrics = 'auc',
                as_pandas = True,  # return the results in a pandas dataframe
                seed = 123)

### importance

In [ ]:
xgb.plot_importance(xgb_clf)
plt.rcParams['figure.figsize'] = [6, 4]
plt.show()